# Objectifs apm

* ~~capture multiples photos sur une durée précise~~ capture video ok -> envoi numpy array au cnn
* ~~se séparer de l'écriture (=enregistrement) en local de ces photos (pas de photo.jpeg en local)~~ os.remove fonctionnel
* ~~parvenir à écrire de multiples frames ou les regrouper dans un batch à passer au cnn (sans écrire le batch en local)~~ inclus en vidéo
* ~~avoir un simple script module_1 qui prenne en input la photo, et la restitue en output~~ ok dummycnn.py
* ignorer deepface: fouiller les outils openCV pour détecter la présence d'un visage
* supprimer l'enregistrement vidéo et simplifier la cellule openCV
* collectiviser les réponses CNN, extraire la plus fréquente, envoyer au LLM cette réponse AVEC une limite dans le temps de nb de requêtes

In [1]:
import numpy
import cv2
import os
from PIL import Image

In [2]:
import testcnn
cap = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 5.0, (640,480))

while( cap.isOpened() ):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.flip(frame,1)
        out.write(frame)
        pilimage = Image.fromarray(frame)
        cnnresponse = testcnn.testrestitution(pilimage)
        print(cnnresponse)
        cv2.imshow('frame' , frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()
os.remove(r"C:\Users\maria\Desktop\dsfs_ft_35\00_Projets_certif\bloc6_lead-data-project_finalproject\jedhaproject_tdahdetection\app\output.avi")

doubt
annoyance
annoyance
disconnection
other
disconnection
annoyance
fatigue
annoyance
fatigue
pain
annoyance
annoyance
disquietment
doubt
disquietment
other
annoyance
disconnection
disconnection
doubt
annoyance
annoyance
disquietment
other
disquietment
fatigue
pain
disconnection
doubt
disquietment
pain
disconnection
doubt
other
disquietment
annoyance
other
annoyance
other
other
fatigue
other
fatigue
disquietment
disconnection
pain
annoyance
disquietment
annoyance
pain
other
doubt
annoyance
other
doubt
annoyance
annoyance
pain
disquietment
doubt
disconnection
other
other
doubt
other
disquietment
doubt
disconnection
other
doubt
doubt
disconnection
doubt
doubt
fatigue
annoyance
annoyance
disquietment
other
fatigue
disquietment
doubt
disconnection
pain
doubt
doubt
annoyance
disquietment
other
other
disquietment
pain
fatigue
annoyance
doubt
annoyance
disquietment
doubt
annoyance
pain
other
annoyance
doubt
disconnection
annoyance
disconnection
fatigue
annoyance
annoyance
annoyance
pain
ann

In [24]:
# Initialize webcam (0 = default camera)
cam = cv2.VideoCapture(0)

# Capture one frame
ret, frame = cam.read()

if ret:
    cv2.imwrite("captured_image.jpeg", frame)  
else:
    print("Failed to capture image.")

cam.release()

In [25]:
#removed lines: imshow shows pending window with result, waitKey & destroyWindow functionalities to close that pending window
#    cv2.imshow("Captured", frame)         
#    cv2.waitKey(0)                      
#    cv2.destroyWindow("Captured")       

In [2]:
%load_ext autoreload
%autoreload 2
import module_1
module_1.random_nb_call()

2

In [ ]:
# --- 2. Transformations d'images ---
# Transformations pour l'entraînement (augmentation des données)
data_transforms = {
    'train': transforms.Compose([
        transforms.RGB(),
        transforms.RandomResizedCrop(224), # Recadrage aléatoire et redimensionnement à 224x224
        transforms.RandomHorizontalFlip(), # Retournement horizontal aléatoire
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Jitter de couleur
        transforms.ToTensor(), # Convertir en Tensor PyTorch (met les pixels entre 0 et 1)
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalisation (moyenne et std ImageNet)
    ]),
    # Transformations pour la validation/test (juste redimensionnement et normalisation)
    'val': transforms.Compose([
        transforms.RGB(),
        transforms.Resize(256), # Redimensionner le plus petit côté à 256
        transforms.CenterCrop(224), # Centre Crop à 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}